In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.3 MB/s eta 0:00:00


In [19]:
# from sentence_transformers import SentenceTransformer
# import torch
# from transformers import AutoProcessor, AutoModel
# from PIL import Image
import pandas as pd
import polars as pl
# from tqdm import tqdm

import numpy as np
from sklearn.model_selection import RandomizedSearchCV, train_test_split, StratifiedKFold
from sklearn.metrics import average_precision_score, make_scorer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
# from sklearn.ensemble import StackingClassifier
# from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
# import xgboost as xgb
# from catboost import CatBoostClassifier, Pool

In [3]:
train_df = pl.scan_parquet('/content/drive/MyDrive/wb/train_simple.parquet')
train_df_pd = train_df.collect().to_pandas()
train_df_pd

,id,id1,subjectname1,title1,description1,id2,subjectname2,title2,description2,target,subjectname_sim,title_sim,description_sim
0,a7c696cf3301e2b9c711,a7c696cf33,Кремы,Крем для лица интенсивно увлажняющий с мочевин...,"Лёгкий, быстро впитывающийся крем с мочевиной ...",01e2b9c711,Кремы,Крем для лица интенсивно увлажняющий с мочевин...,"Intensive Moisture с мочевиной (10%) Лёгкий, ...",True,1.000000,0.986537,0.772320
1,5cba57d7b615d3377eaf,5cba57d7b6,Маски косметические,Маска для волос увлажняющая 350мл,"Подарите своим волосам заботу, которую они зас...",15d3377eaf,Маски косметические,Парфюмированная маска для сухих волос Movet 35...,Парфюмированная маска для волос Lador Keratin ...,True,1.000000,0.680832,0.753266
2,151f306a64446b7e8ed0,151f306a64,Спреи,Парфюмированный спрей 100 мл,Парфюмированный спрей для тела 100 мл.Спрей ид...,446b7e8ed0,Спреи,Лосьон спрей манго и франжипани 100 мл,"Легкий спрей для тела Avon ""Тропическое путеше...",False,1.000000,0.824037,0.466828
3,7a6ed390ecef983199f2,7a6ed390ec,Шампуни,Шампунь и гель для душа для детей 250 мл,Шампунь-гель для волос и тела детский Bambolin...,ef983199f2,Шампуни,Шампунь и гель для душа для детей,Шампунь и гель для душа для детей — это идеаль...,False,1.000000,0.899106,0.841057
4,a48a0f884afc2df6cfa2,a48a0f884a,Гели,Средство гель для купания малыша и детский шам...,Средство гель для купания малыша и детский шам...,fc2df6cfa2,Гели,"Средство для купания малыша и шампунь 2 в 1, 4...",Средство для купания малыша и шампунь 2 в 1 Aq...,True,1.000000,0.810535,0.916536
...,...,...,...,...,...,...,...,...,...,...,...,...,...
239744,11f39ee08ae43640d407,11f39ee08a,Духи,Kenzo L'Eau Par Femme Кензо ля пур ле пар фемм...,L'Eau Par Kenzo женский аромат и принадлежит с...,e43640d407,Духи,LEau Par Kenzo pour femme,L'Eau Par Kenzo женский аромат и принадлежит с...,True,1.000000,0.594768,1.000000
239745,af28f573e8aaca981768,af28f573e8,Гели,Гель для душа Белая орхидея 400 мл,"AVE Гель для душа Белая орхидея, 400 мл. Гель ...",aaca981768,Гели,Гель для душа с экстрактом орхидеи,Гель для душа AVE с экстрактом орхидеи\n\nПогр...,True,1.000000,0.764349,0.876597
239746,85ec4648bda46a9a6a98,85ec4648bd,Дезодоранты,Tiziana Terenzi Kirke Тизиана Кирки Парфюмерны...,Tiziana Terenzi Kirke - сочащийся сладостью и...,a46a9a6a98,Духи,Масляные духи kirke tiziana terenzil 10ml,Масляные духи Kirke tiziana terenzi от бренда ...,False,0.573682,0.694591,0.865917
239747,f4d6a8047342a7c81a64,f4d6a80473,Туши,Тушь для ресниц PUSH-UP! Экстремальный Объем 1...,Eveline Тушь для ресниц Потрясающий эффект PUS...,42a7c81a64,Туши,Тушь для ресниц Extension Volume,В состав туши входят натуральный пчелиный воск...,True,1.000000,0.597250,0.713343


In [4]:
test_df = pl.scan_parquet('/content/drive/MyDrive/wb/test_simple.parquet')
test_df_pd = test_df.collect().to_pandas()
test_df_pd

,id,id1,subjectname1,title1,description1,id2,subjectname2,title2,description2,subjectname_sim,title_sim,description_sim
0,82e68c9404e40d1c3577,82e68c9404,Духи,No 4 Apres L'Amour,No 4 Apres L'Amour — нишевая парфюмерная вода ...,e40d1c3577,Духи,Духи Томас Космала 4 Апрес Лямур Древесные Фуж...,Eau de Parfum Парфюмерная вода Après l’Amou...,1.000000,0.257982,0.858906
1,3f9db954a1926f8e58e8,3f9db954a1,Духи,ELIZABETH ARDEN Green Tea цитрусовый свежий,"ELIZABETH ARDEN Green Tea - роскошный аромат, ...",926f8e58e8,Духи,Духи женские Elizabeth Arden Green Tea 100мл,Духи женские Elizabeth Arden Green Tea 100мл —...,1.000000,0.838960,0.912406
2,a46ae482ef6932a2b95d,a46ae482ef,Спреи,Vitamino Color Термозащитный спрей для окрашен...,Продлить эффект салонного окрашивания возможно...,6932a2b95d,Спреи,Vitamino Color Термозащита спрей для окрашенны...,Термозащитный спрей-уход для окрашенных волос ...,1.000000,0.989620,0.698489
3,eba9fec0558ac6f80602,eba9fec055,Шампуни,"Fiber infusion , Шампунь с кератином, 250мл",Шампунь с кератином Fiber Infusion мгновенно в...,8ac6f80602,Шампуни,Шампунь с кератином Fiber Infusion 250 мл,Шампунь для волос Londa Professional Fiber Inf...,1.000000,0.961485,0.879102
4,2725e3d5cd74a31d08a7,2725e3d5cd,Пудры,Пудра компактная для лица матирующая тон 03,Пудра с легкой инновационной текстурой обеспеч...,74a31d08a7,Пудры,"Пудра Skinvisible Carcass, тон 03 бежевый, 4,2 г",Пудра с легкой инновационной текстурой обеспеч...,1.000000,0.532757,0.999838
...,...,...,...,...,...,...,...,...,...,...,...,...
59683,645c6dc3143c9bd865f7,645c6dc314,Тени,"Тени жидкие матовые Matt tint waterproof 12H, ...",Тени жидкие матовые LUXVISAGE Matt tint waterp...,3c9bd865f7,Тени,"Тени для век жидкие, Matt In Waterprof 12H, то...","Тени для век Luxvisage жидкие, Matt In Waterpr...",1.000000,0.776864,0.961291
59684,1e0499f3902993d6f080,1e0499f390,Туалетная вода,Арабские духи женские Yara Tous 100мл,Роскошные женские духи премиум-класса Lattafa...,2993d6f080,Духи,Арабские духи Yara Tous 100 мл. Латтафа Яра Тоус,"Lattafa Yara Tous — яркий, солнечный и жизнера...",0.420592,0.867039,0.808429
59685,d4dca3fa2671a4cff5ef,d4dca3fa26,Патчи,Гидрогелевые патчи для глаз 60 шт.,Zozu Гидрогелевые патчи для глаз с экстрактом ...,71a4cff5ef,Патчи,Антивозрастные патчи для глаз от отеков и морщин,Увлажняющие гидрогелевые патчи с экстрактом ав...,1.000000,0.689708,0.791846
59686,a5692290e71478eeb16d,a5692290e7,Краски для волос,Краска Studio 9.23 Очень светлый бежевый перла...,Крем-краска для волос со сбалансированной сист...,1478eeb16d,Краски для волос,Крем краска для волос Капус Студио 9.23,Профессиональная краска Kapous очень светлый б...,1.000000,0.647164,0.806553


In [5]:
categorical_features = ['subjectname1', 'subjectname2']
text_features = ['title1', 'title2', 'description1', 'description2']

## Косинусная близость текстов

In [ ]:
# id1 = train_df_pd['id1'].tolist()
# id2 = train_df_pd['id2'].tolist()
subjectname1 = train_df_pd['subjectname1'].tolist()
subjectname2 = train_df_pd['subjectname2'].tolist()
title1 = train_df_pd['title1'].tolist()
title2 = train_df_pd['title2'].tolist()
description1 = train_df_pd['description1'].tolist()
description2 = train_df_pd['description2'].tolist()

subjectname_sim = []
title_sim = []
description_sim = []
# image_sim = []

In [ ]:
text_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def get_text_embedding(text: str):
  emb = text_model.encode(text, convert_to_tensor=True, normalize_embeddings=True)
  return emb



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def cosine_similarity(emb1, emb2):
  return torch.nn.functional.cosine_similarity(
    emb1.unsqueeze(0),
    emb2.unsqueeze(0)
  ).item()

In [ ]:
def get_text_sim(text1, text2):
  t1 = get_text_embedding(text1)
  t2 = get_text_embedding(text2)
  text_sim = cosine_similarity(t1, t2)
  return text_sim

In [ ]:
for i in tqdm(range(239749)):
  subjectname_sim.append(get_text_sim(subjectname1[i], subjectname2[i]))
  title_sim.append(get_text_sim(title1[i], title2[i]))
  description_sim.append(get_text_sim(description1[i], description2[i]))


100%|██████████| 239749/239749 [2:48:59<00:00, 23.64it/s]


In [ ]:
train_df_pd['subjectname_sim'] = subjectname_sim
train_df_pd['title_sim'] = title_sim
train_df_pd['description_sim'] = description_sim

train_df_pd.to_parquet('/content/drive/MyDrive/wb/train_simple.parquet')
train_df_pd

,id,id1,subjectname1,title1,description1,id2,subjectname2,title2,description2,target,subjectname_sim,title_sim,description_sim
0,a7c696cf3301e2b9c711,a7c696cf33,Кремы,Крем для лица интенсивно увлажняющий с мочевин...,"Лёгкий, быстро впитывающийся крем с мочевиной ...",01e2b9c711,Кремы,Крем для лица интенсивно увлажняющий с мочевин...,"Intensive Moisture с мочевиной (10%) Лёгкий, ...",True,1.000000,0.986537,0.772320
1,5cba57d7b615d3377eaf,5cba57d7b6,Маски косметические,Маска для волос увлажняющая 350мл,"Подарите своим волосам заботу, которую они зас...",15d3377eaf,Маски косметические,Парфюмированная маска для сухих волос Movet 35...,Парфюмированная маска для волос Lador Keratin ...,True,1.000000,0.680832,0.753266
2,151f306a64446b7e8ed0,151f306a64,Спреи,Парфюмированный спрей 100 мл,Парфюмированный спрей для тела 100 мл.Спрей ид...,446b7e8ed0,Спреи,Лосьон спрей манго и франжипани 100 мл,"Легкий спрей для тела Avon ""Тропическое путеше...",False,1.000000,0.824037,0.466828
3,7a6ed390ecef983199f2,7a6ed390ec,Шампуни,Шампунь и гель для душа для детей 250 мл,Шампунь-гель для волос и тела детский Bambolin...,ef983199f2,Шампуни,Шампунь и гель для душа для детей,Шампунь и гель для душа для детей — это идеаль...,False,1.000000,0.899106,0.841057
4,a48a0f884afc2df6cfa2,a48a0f884a,Гели,Средство гель для купания малыша и детский шам...,Средство гель для купания малыша и детский шам...,fc2df6cfa2,Гели,"Средство для купания малыша и шампунь 2 в 1, 4...",Средство для купания малыша и шампунь 2 в 1 Aq...,True,1.000000,0.810535,0.916536
...,...,...,...,...,...,...,...,...,...,...,...,...,...
239744,11f39ee08ae43640d407,11f39ee08a,Духи,Kenzo L'Eau Par Femme Кензо ля пур ле пар фемм...,L'Eau Par Kenzo женский аромат и принадлежит с...,e43640d407,Духи,LEau Par Kenzo pour femme,L'Eau Par Kenzo женский аромат и принадлежит с...,True,1.000000,0.594768,1.000000
239745,af28f573e8aaca981768,af28f573e8,Гели,Гель для душа Белая орхидея 400 мл,"AVE Гель для душа Белая орхидея, 400 мл. Гель ...",aaca981768,Гели,Гель для душа с экстрактом орхидеи,Гель для душа AVE с экстрактом орхидеи\n\nПогр...,True,1.000000,0.764349,0.876597
239746,85ec4648bda46a9a6a98,85ec4648bd,Дезодоранты,Tiziana Terenzi Kirke Тизиана Кирки Парфюмерны...,Tiziana Terenzi Kirke - сочащийся сладостью и...,a46a9a6a98,Духи,Масляные духи kirke tiziana terenzil 10ml,Масляные духи Kirke tiziana terenzi от бренда ...,False,0.573682,0.694591,0.865917
239747,f4d6a8047342a7c81a64,f4d6a80473,Туши,Тушь для ресниц PUSH-UP! Экстремальный Объем 1...,Eveline Тушь для ресниц Потрясающий эффект PUS...,42a7c81a64,Туши,Тушь для ресниц Extension Volume,В состав туши входят натуральный пчелиный воск...,True,1.000000,0.597250,0.713343


In [ ]:
test_df = pl.scan_parquet('/content/drive/MyDrive/wb/beauty_test.parquet').drop(["characteristics1", "characteristics2", "parentname1", "parentname2"])
test_df_pd = test_df.collect().to_pandas()
test_df_pd

,id,id1,subjectname1,title1,description1,id2,subjectname2,title2,description2
0,82e68c9404e40d1c3577,82e68c9404,Духи,No 4 Apres L'Amour,No 4 Apres L'Amour — нишевая парфюмерная вода ...,e40d1c3577,Духи,Духи Томас Космала 4 Апрес Лямур Древесные Фуж...,Eau de Parfum Парфюмерная вода Après l’Amou...
1,3f9db954a1926f8e58e8,3f9db954a1,Духи,ELIZABETH ARDEN Green Tea цитрусовый свежий,"ELIZABETH ARDEN Green Tea - роскошный аромат, ...",926f8e58e8,Духи,Духи женские Elizabeth Arden Green Tea 100мл,Духи женские Elizabeth Arden Green Tea 100мл —...
2,a46ae482ef6932a2b95d,a46ae482ef,Спреи,Vitamino Color Термозащитный спрей для окрашен...,Продлить эффект салонного окрашивания возможно...,6932a2b95d,Спреи,Vitamino Color Термозащита спрей для окрашенны...,Термозащитный спрей-уход для окрашенных волос ...
3,eba9fec0558ac6f80602,eba9fec055,Шампуни,"Fiber infusion , Шампунь с кератином, 250мл",Шампунь с кератином Fiber Infusion мгновенно в...,8ac6f80602,Шампуни,Шампунь с кератином Fiber Infusion 250 мл,Шампунь для волос Londa Professional Fiber Inf...
4,2725e3d5cd74a31d08a7,2725e3d5cd,Пудры,Пудра компактная для лица матирующая тон 03,Пудра с легкой инновационной текстурой обеспеч...,74a31d08a7,Пудры,"Пудра Skinvisible Carcass, тон 03 бежевый, 4,2 г",Пудра с легкой инновационной текстурой обеспеч...
...,...,...,...,...,...,...,...,...,...
59683,645c6dc3143c9bd865f7,645c6dc314,Тени,"Тени жидкие матовые Matt tint waterproof 12H, ...",Тени жидкие матовые LUXVISAGE Matt tint waterp...,3c9bd865f7,Тени,"Тени для век жидкие, Matt In Waterprof 12H, то...","Тени для век Luxvisage жидкие, Matt In Waterpr..."
59684,1e0499f3902993d6f080,1e0499f390,Туалетная вода,Арабские духи женские Yara Tous 100мл,Роскошные женские духи премиум-класса Lattafa...,2993d6f080,Духи,Арабские духи Yara Tous 100 мл. Латтафа Яра Тоус,"Lattafa Yara Tous — яркий, солнечный и жизнера..."
59685,d4dca3fa2671a4cff5ef,d4dca3fa26,Патчи,Гидрогелевые патчи для глаз 60 шт.,Zozu Гидрогелевые патчи для глаз с экстрактом ...,71a4cff5ef,Патчи,Антивозрастные патчи для глаз от отеков и морщин,Увлажняющие гидрогелевые патчи с экстрактом ав...
59686,a5692290e71478eeb16d,a5692290e7,Краски для волос,Краска Studio 9.23 Очень светлый бежевый перла...,Крем-краска для волос со сбалансированной сист...,1478eeb16d,Краски для волос,Крем краска для волос Капус Студио 9.23,Профессиональная краска Kapous очень светлый б...


In [ ]:
subjectname1 = test_df_pd['subjectname1'].tolist()
subjectname2 = test_df_pd['subjectname2'].tolist()
title1 = test_df_pd['title1'].tolist()
title2 = test_df_pd['title2'].tolist()
description1 = test_df_pd['description1'].tolist()
description2 = test_df_pd['description2'].tolist()

subjectname_sim = []
title_sim = []
description_sim = []

In [ ]:
for i in tqdm(range(59688)):
  subjectname_sim.append(get_text_sim(subjectname1[i], subjectname2[i]))
  title_sim.append(get_text_sim(title1[i], title2[i]))
  description_sim.append(get_text_sim(description1[i], description2[i]))

100%|██████████| 59688/59688 [43:08<00:00, 23.06it/s]


In [ ]:
test_df_pd['subjectname_sim'] = subjectname_sim
test_df_pd['title_sim'] = title_sim
test_df_pd['description_sim'] = description_sim

test_df_pd.to_parquet('/content/drive/MyDrive/wb/test_simple.parquet')
test_df_pd

,id,id1,subjectname1,title1,description1,id2,subjectname2,title2,description2,subjectname_sim,title_sim,description_sim
0,82e68c9404e40d1c3577,82e68c9404,Духи,No 4 Apres L'Amour,No 4 Apres L'Amour — нишевая парфюмерная вода ...,e40d1c3577,Духи,Духи Томас Космала 4 Апрес Лямур Древесные Фуж...,Eau de Parfum Парфюмерная вода Après l’Amou...,1.000000,0.257982,0.858906
1,3f9db954a1926f8e58e8,3f9db954a1,Духи,ELIZABETH ARDEN Green Tea цитрусовый свежий,"ELIZABETH ARDEN Green Tea - роскошный аромат, ...",926f8e58e8,Духи,Духи женские Elizabeth Arden Green Tea 100мл,Духи женские Elizabeth Arden Green Tea 100мл —...,1.000000,0.838960,0.912406
2,a46ae482ef6932a2b95d,a46ae482ef,Спреи,Vitamino Color Термозащитный спрей для окрашен...,Продлить эффект салонного окрашивания возможно...,6932a2b95d,Спреи,Vitamino Color Термозащита спрей для окрашенны...,Термозащитный спрей-уход для окрашенных волос ...,1.000000,0.989620,0.698489
3,eba9fec0558ac6f80602,eba9fec055,Шампуни,"Fiber infusion , Шампунь с кератином, 250мл",Шампунь с кератином Fiber Infusion мгновенно в...,8ac6f80602,Шампуни,Шампунь с кератином Fiber Infusion 250 мл,Шампунь для волос Londa Professional Fiber Inf...,1.000000,0.961485,0.879102
4,2725e3d5cd74a31d08a7,2725e3d5cd,Пудры,Пудра компактная для лица матирующая тон 03,Пудра с легкой инновационной текстурой обеспеч...,74a31d08a7,Пудры,"Пудра Skinvisible Carcass, тон 03 бежевый, 4,2 г",Пудра с легкой инновационной текстурой обеспеч...,1.000000,0.532757,0.999838
...,...,...,...,...,...,...,...,...,...,...,...,...
59683,645c6dc3143c9bd865f7,645c6dc314,Тени,"Тени жидкие матовые Matt tint waterproof 12H, ...",Тени жидкие матовые LUXVISAGE Matt tint waterp...,3c9bd865f7,Тени,"Тени для век жидкие, Matt In Waterprof 12H, то...","Тени для век Luxvisage жидкие, Matt In Waterpr...",1.000000,0.776864,0.961291
59684,1e0499f3902993d6f080,1e0499f390,Туалетная вода,Арабские духи женские Yara Tous 100мл,Роскошные женские духи премиум-класса Lattafa...,2993d6f080,Духи,Арабские духи Yara Tous 100 мл. Латтафа Яра Тоус,"Lattafa Yara Tous — яркий, солнечный и жизнера...",0.420592,0.867039,0.808429
59685,d4dca3fa2671a4cff5ef,d4dca3fa26,Патчи,Гидрогелевые патчи для глаз 60 шт.,Zozu Гидрогелевые патчи для глаз с экстрактом ...,71a4cff5ef,Патчи,Антивозрастные патчи для глаз от отеков и морщин,Увлажняющие гидрогелевые патчи с экстрактом ав...,1.000000,0.689708,0.791846
59686,a5692290e71478eeb16d,a5692290e7,Краски для волос,Краска Studio 9.23 Очень светлый бежевый перла...,Крем-краска для волос со сбалансированной сист...,1478eeb16d,Краски для волос,Крем краска для волос Капус Студио 9.23,Профессиональная краска Kapous очень светлый б...,1.000000,0.647164,0.806553


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Catboost

In [6]:
# Разделение на признаки и целевую переменную
X = train_df_pd.drop(columns=["target", "id", "id1", "id2"])
y = train_df_pd['target']

# Разделение на train/val
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.1, random_state=42, stratify=y
)

X_test = test_df_pd.drop(columns=["id", "id1", "id2"])


In [7]:
train_pool = Pool(
    data=X_train,
    label=y_train,
    cat_features=categorical_features,
    text_features=text_features
)

val_pool = Pool(
    data=X_val,
    label=y_val,
    cat_features=categorical_features,
    text_features=text_features
)

test_pool = Pool(
    data=X_test,
    cat_features=categorical_features,
    text_features=text_features
)

### Подбор гиперпараметров

In [ ]:
average_precision_scorer = make_scorer(
    average_precision_score,
    response_method='predict_proba',
    greater_is_better=True
)

In [ ]:
base_model = CatBoostClassifier(
    cat_features=categorical_features,
    text_features=text_features,
    random_seed=42,
    verbose=0,
    task_type='CPU',
    eval_metric='PRAUC',
    early_stopping_rounds=50
)

In [ ]:
param_dist = {
    'iterations': [100],
    'depth': [3, 6, 8],
    'learning_rate': [0.01, 0.1, 0.3],
    'l2_leaf_reg': [1, 5, 9],
    'bagging_temperature': [0, 1.0, 2.0],
    'random_strength': [1, 3, 5],
    'border_count': [32, 150, 254],
    'min_data_in_leaf': [1, 10, 20],
    'grow_policy': ['SymmetricTree', 'Depthwise', 'Lossguide'],
    'bootstrap_type': ['Bayesian']
}

In [ ]:
random_search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_dist,
    n_iter=3,                     # Количество случайных комбинаций
    scoring=average_precision_scorer,  # Наша метрика
    cv=3,                          # Количество фолдов кросс-валидации
    n_jobs=1,                      # Для Pool используем 1 job
    verbose=2,                     # Детальный вывод прогресса
    random_state=42,               # Для воспроизводимости
)

In [ ]:
random_search.fit(
    X_train, y_train,           # Данные для обучения
    eval_set=(X_val, y_val),    # Данные для валидации
    verbose=0                   # Отключаем вывод CatBoost
)

print("\nЛУЧШИЕ ПАРАМЕТРЫ:")
for param, value in random_search.best_params_.items():
    print(f"  {param}: {value}")

# print(f"\nЛучший Average Precision Score (кросс-валидация): {random_search.best_score_:.4f}")

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] END bagging_temperature=0, bootstrap_type=Bayesian, border_count=150, depth=3, grow_policy=Depthwise, iterations=100, l2_leaf_reg=5, learning_rate=0.3, min_data_in_leaf=10, random_strength=5; total time= 3.6min
[CV] END bagging_temperature=0, bootstrap_type=Bayesian, border_count=150, depth=3, grow_policy=Depthwise, iterations=100, l2_leaf_reg=5, learning_rate=0.3, min_data_in_leaf=10, random_strength=5; total time= 3.7min
[CV] END bagging_temperature=0, bootstrap_type=Bayesian, border_count=150, depth=3, grow_policy=Depthwise, iterations=100, l2_leaf_reg=5, learning_rate=0.3, min_data_in_leaf=10, random_strength=5; total time= 3.6min
[CV] END bagging_temperature=2.0, bootstrap_type=Bayesian, border_count=150, depth=6, grow_policy=SymmetricTree, iterations=100, l2_leaf_reg=5, learning_rate=0.1, min_data_in_leaf=20, random_strength=5; total time= 4.4min
[CV] END bagging_temperature=2.0, bootstrap_type=Bayesian, border_count

### Лучшая модель

In [8]:
best_model = CatBoostClassifier(
    cat_features=categorical_features,
    text_features=text_features,
    random_seed=42,
    verbose=100,
    task_type='GPU',
    eval_metric='PRAUC',
    early_stopping_rounds=50,
    random_strength = 5,
    min_data_in_leaf = 10,
    learning_rate = 0.3,
    iterations=1500,
    grow_policy = 'Depthwise',
    depth=3,
    border_count = 150,
    bootstrap_type = 'Bayesian',
    bagging_temperature = 0

)

best_model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=100)
y_pred_proba = best_model.predict_proba(X_val)[:, 1]
ap_score = average_precision_score(y_val, y_pred_proba)
print(f"Average Precision на валидации: {ap_score:.4f}")



Default metric period is 5 because PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.8665548	test: 0.8759953	best: 0.8759953 (0)	total: 164ms	remaining: 4m 5s
100:	learn: 0.9084327	test: 0.9116803	best: 0.9116803 (100)	total: 3.24s	remaining: 44.8s
200:	learn: 0.9179966	test: 0.9177328	best: 0.9177328 (200)	total: 6.05s	remaining: 39.1s
300:	learn: 0.9245235	test: 0.9222513	best: 0.9222660 (299)	total: 11.6s	remaining: 46s
400:	learn: 0.9295876	test: 0.9259245	best: 0.9259245 (400)	total: 14.3s	remaining: 39.3s
500:	learn: 0.9338943	test: 0.9284692	best: 0.9284692 (500)	total: 17.2s	remaining: 34.2s
600:	learn: 0.9380220	test: 0.9307040	best: 0.9307040 (600)	total: 19.9s	remaining: 29.8s
700:	learn: 0.9417518	test: 0.9322295	best: 0.9322295 (700)	total: 25.2s	remaining: 28.7s
800:	learn: 0.9449370	test: 0.9336308	best: 0.9336308 (800)	total: 28s	remaining: 24.4s
900:	learn: 0.9477283	test: 0.9349982	best: 0.9350123 (898)	total: 30.8s	remaining: 20.5s
1000:	learn: 0.9502093	test: 0.9360805	best: 0.9361061 (993)	total: 33.5s	remaining: 16.7s
1100:	learn: 0.95

In [9]:
test_prediction = best_model.predict_proba(test_pool)[:, 1]

In [10]:
sub_df = test_df.select("id").collect().to_pandas()
sub_df.loc[:, "y_pred"] = test_prediction
sub_df.to_csv("/content/drive/MyDrive/wb/catboost_submission.csv", index=False)
sub_df.head()

,id,y_pred
0,82e68c9404e40d1c3577,0.197420
1,3f9db954a1926f8e58e8,0.427212
2,a46ae482ef6932a2b95d,0.953303
3,eba9fec0558ac6f80602,0.964975
4,2725e3d5cd74a31d08a7,0.933897


## OHE

In [6]:
# Создаем OneHotEncoder с обработкой неизвестных категорий
ohe = OneHotEncoder(
    sparse_output=False,  # возвращаем обычный массив, не sparse
    handle_unknown='ignore',  # игнорируем неизвестные категории в test
    drop='first'  # можно убрать первый столбец чтобы избежать dummy trap
)

# Применяем к train
X_train_cat = train_df_pd[categorical_features]
X_test_cat = test_df_pd[categorical_features]

# Обучаем encoder на train
ohe.fit(X_train_cat)

# Преобразуем train и test
train_ohe = ohe.transform(X_train_cat)
test_ohe = ohe.transform(X_test_cat)

# Получаем имена столбцов
ohe_columns = ohe.get_feature_names_out(categorical_features)

# Создаем датафреймы с OHE признаками
train_ohe_df = pd.DataFrame(train_ohe, columns=ohe_columns, index=train_df_pd.index)
test_ohe_df = pd.DataFrame(test_ohe, columns=ohe_columns, index=test_df_pd.index)

# Удаляем исходные категориальные столбцы
train_df_processed = train_df_pd.drop(categorical_features, axis=1)
test_df_processed = test_df_pd.drop(categorical_features, axis=1)

# Объединяем с OHE признаками
train_final = pd.concat([train_df_processed, train_ohe_df], axis=1).drop(text_features, axis=1)
test_final = pd.concat([test_df_processed, test_ohe_df], axis=1).drop(text_features, axis=1)

print(f"Train shape: {train_final.shape}")
print(f"Test shape: {test_final.shape}")

/usr/local/lib/python3.12/dist-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Train shape: (239749, 267)
Test shape: (59688, 266)


In [7]:
train_final

,id,id1,id2,target,subjectname_sim,title_sim,description_sim,subjectname1_Дезодоранты,subjectname1_Духи,subjectname1_Зубные пасты,...,subjectname2_Шампуни,subjectname2_Шампуни сухие,subjectname2_Шапочки косметические,subjectname2_Шиммер для ванн,subjectname2_Щетки косметические,subjectname2_Экстракты для купания,subjectname2_Эликсиры,subjectname2_Эмульсии,subjectname2_Эссенции,subjectname2_Эфирные масла
0,a7c696cf3301e2b9c711,a7c696cf33,01e2b9c711,True,1.000000,0.986537,0.772320,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5cba57d7b615d3377eaf,5cba57d7b6,15d3377eaf,True,1.000000,0.680832,0.753266,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,151f306a64446b7e8ed0,151f306a64,446b7e8ed0,False,1.000000,0.824037,0.466828,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7a6ed390ecef983199f2,7a6ed390ec,ef983199f2,False,1.000000,0.899106,0.841057,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,a48a0f884afc2df6cfa2,a48a0f884a,fc2df6cfa2,True,1.000000,0.810535,0.916536,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239744,11f39ee08ae43640d407,11f39ee08a,e43640d407,True,1.000000,0.594768,1.000000,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
239745,af28f573e8aaca981768,af28f573e8,aaca981768,True,1.000000,0.764349,0.876597,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
239746,85ec4648bda46a9a6a98,85ec4648bd,a46a9a6a98,False,0.573682,0.694591,0.865917,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
239747,f4d6a8047342a7c81a64,f4d6a80473,42a7c81a64,True,1.000000,0.597250,0.713343,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
test_final

,id,id1,id2,subjectname_sim,title_sim,description_sim,subjectname1_Дезодоранты,subjectname1_Духи,subjectname1_Зубные пасты,subjectname1_Косметические карандаши,...,subjectname2_Шампуни,subjectname2_Шампуни сухие,subjectname2_Шапочки косметические,subjectname2_Шиммер для ванн,subjectname2_Щетки косметические,subjectname2_Экстракты для купания,subjectname2_Эликсиры,subjectname2_Эмульсии,subjectname2_Эссенции,subjectname2_Эфирные масла
0,82e68c9404e40d1c3577,82e68c9404,e40d1c3577,1.000000,0.257982,0.858906,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3f9db954a1926f8e58e8,3f9db954a1,926f8e58e8,1.000000,0.838960,0.912406,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,a46ae482ef6932a2b95d,a46ae482ef,6932a2b95d,1.000000,0.989620,0.698489,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,eba9fec0558ac6f80602,eba9fec055,8ac6f80602,1.000000,0.961485,0.879102,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2725e3d5cd74a31d08a7,2725e3d5cd,74a31d08a7,1.000000,0.532757,0.999838,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59683,645c6dc3143c9bd865f7,645c6dc314,3c9bd865f7,1.000000,0.776864,0.961291,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59684,1e0499f3902993d6f080,1e0499f390,2993d6f080,0.420592,0.867039,0.808429,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59685,d4dca3fa2671a4cff5ef,d4dca3fa26,71a4cff5ef,1.000000,0.689708,0.791846,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59686,a5692290e71478eeb16d,a5692290e7,1478eeb16d,1.000000,0.647164,0.806553,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
X = train_final.drop(columns=["target", "id", "id1", "id2"])
y = train_final['target']

# Разделение на train/val
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.1, random_state=42, stratify=y
)

X_test = test_final.drop(columns=["id", "id1", "id2"])

## XGBoost

### Подбор модели

In [7]:
average_precision_scorer = make_scorer(
    average_precision_score,
    response_method='predict_proba',
    greater_is_better=True
)

In [8]:
base_model = xgb.XGBClassifier(
    objective='binary:logistic',  # для бинарной классификации
    eval_metric='aucpr',         # можно также использовать 'auc', 'aucpr'
    use_label_encoder=False,
    random_state=42,
    n_jobs=-1,                     # использовать все ядра
    verbosity=0                    # отключить вывод
)

In [9]:
simple_param_dist = {
    'n_estimators': [100],
    'max_depth': [4, 6, 8],
    'learning_rate': [0.01, 0.1, 0.3],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.3],
    'reg_lambda': [1, 3, 5]
}

In [11]:
random_search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=simple_param_dist,
    n_iter=30,
    scoring=average_precision_scorer,
    cv=3,                            # количество фолдов
    n_jobs=-1,                       # использовать все ядра
    verbose=3,                       # детальный вывод
    random_state=42,
    refit=False,                      # переобучить лучшую модель на всех данных
    return_train_score=True          # сохранить результаты на train
)

In [12]:
random_search.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],  # для ранней остановки в XGBoost
    verbose=0
)

print("\nЛУЧШИЕ ПАРАМЕТРЫ:")
for param, value in random_search.best_params_.items():
    print(f"  {param}: {value}")

Fitting 3 folds for each of 30 candidates, totalling 90 fits

ЛУЧШИЕ ПАРАМЕТРЫ:
  subsample: 0.8
  reg_lambda: 5
  n_estimators: 100
  max_depth: 8
  learning_rate: 0.3
  gamma: 0.3
  colsample_bytree: 0.9


### Лучшая модель

In [35]:
xgb.set_config(verbosity=1)

best_model = xgb.XGBClassifier(
    objective='binary:logistic',  # для бинарной классификации
    eval_metric='aucpr',
    use_label_encoder=False,
    random_state=42,
    n_jobs=-1,                     # использовать все ядра
    subsample=0.8,
    reg_lambda=5,
    n_estimators=250,
    max_depth=8,
    learning_rate=0.3,
    gamma=0.3,
    colsample_bytree=0.9,
    # verbosity=0
    # tree_method='hist',
    # early_stopping_rounds=50,
)

In [36]:
best_model.fit(X_train, y_train, verbose=0)
y_pred_proba = best_model.predict_proba(X_val)[:, 1]
ap_score = average_precision_score(y_val, y_pred_proba)
print(f"Average Precision на валидации: {ap_score:.4f}")

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [09:54:47] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Average Precision на валидации: 0.8158


In [37]:
test_prediction = best_model.predict_proba(X_test)[:, 1]
sub_df = test_df.select("id").collect().to_pandas()
sub_df.loc[:, "y_pred"] = test_prediction
sub_df.to_csv("/content/drive/MyDrive/wb/xgboost_submission.csv", index=False)
sub_df.head()

,id,y_pred
0,82e68c9404e40d1c3577,0.091195
1,3f9db954a1926f8e58e8,0.392399
2,a46ae482ef6932a2b95d,0.973472
3,eba9fec0558ac6f80602,0.842933
4,2725e3d5cd74a31d08a7,0.813849


In [38]:
sub_df

,id,y_pred
0,82e68c9404e40d1c3577,0.091195
1,3f9db954a1926f8e58e8,0.392399
2,a46ae482ef6932a2b95d,0.973472
3,eba9fec0558ac6f80602,0.842933
4,2725e3d5cd74a31d08a7,0.813849
...,...,...
59683,645c6dc3143c9bd865f7,0.509472
59684,1e0499f3902993d6f080,0.602464
59685,d4dca3fa2671a4cff5ef,0.322976
59686,a5692290e71478eeb16d,0.581878


## StackingClassifier

In [50]:
xgb.set_config(verbosity=0)

xgb_best_model = xgb.XGBClassifier(
    objective='binary:logistic',  # для бинарной классификации
    eval_metric='aucpr',
    use_label_encoder=False,
    n_jobs=-1,                     # использовать все ядра
    subsample=0.8,
    reg_lambda=5,
    n_estimators=250,
    max_depth=8,
    learning_rate=0.3,
    gamma=0.3,
    colsample_bytree=0.9
)

cb_best_model = CatBoostClassifier(
    verbose=False,
    random_strength = 5,
    min_data_in_leaf = 10,
    grow_policy = 'Depthwise',
    depth=3,
    border_count = 150,
    bootstrap_type = 'Bayesian',
    bagging_temperature = 0,
    learning_rate = 0.3,
    task_type='CPU',
    iterations=1000,
    eval_metric='PRAUC',
)

estimators = [
    ('xgb', xgb_best_model),
    ('cat', cb_best_model)
]

model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())



In [51]:
model.fit(X_train, y_train)
y_pred_proba = model.predict_proba(X_val)[:, 1]
ap_score = average_precision_score(y_val, y_pred_proba)
print(f"Average Precision на валидации: {ap_score:.4f}")

Average Precision на валидации: 0.8177


In [52]:
test_prediction = model.predict_proba(X_test)[:, 1]
sub_df = test_df.select("id").collect().to_pandas()
sub_df.loc[:, "y_pred"] = test_prediction
sub_df.to_csv("/content/drive/MyDrive/wb/stckcls_submission.csv", index=False)
sub_df.head()

,id,y_pred
0,82e68c9404e40d1c3577,0.188695
1,3f9db954a1926f8e58e8,0.462838
2,a46ae482ef6932a2b95d,0.905258
3,eba9fec0558ac6f80602,0.808175
4,2725e3d5cd74a31d08a7,0.854211


## MLP

### Подбор модели

In [13]:
average_precision_scorer = make_scorer(
    average_precision_score,
    response_method='predict_proba',  # MLP возвращает вероятности
    greater_is_better=True
)

In [14]:
base_mlp = MLPClassifier(
    random_state=42,
    verbose=0,  # отключаем вывод обучения
    early_stopping=True,  # ранняя остановка
    validation_fraction=0.1,  # доля данных для валидации при early_stopping
    n_iter_no_change=10,  # остановка если нет улучшения 10 эпох
    tol=1e-4  # порог для сходимости
)

In [23]:
simple_param_dist = {
    'hidden_layer_sizes': [(10,), (10, 10), (20, 10)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001, 0.001, 0.01],
    # 'batch_size': [64, 96, 128],
    'max_iter': [10],
    'learning_rate_init': [0.001, 0.005, 0.01]
}

In [24]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('mlp', base_mlp)
])

simple_param_dist_pipeline = {
    'mlp__' + key: value for key, value in simple_param_dist.items()
}

In [27]:
cv_strategy = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

random_search_mlp = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=simple_param_dist_pipeline,
    n_iter=30,  # количество случайных комбинаций
    scoring=average_precision_scorer,
    cv=cv_strategy,
    n_jobs=-1,  # использовать все ядра
    verbose=3,  # детальный вывод
    random_state=42,
    refit=True,
    return_train_score=True,
    error_score='raise'
)

In [28]:
random_search_mlp.fit(X_train, y_train)

print("\nЛУЧШИЕ ПАРАМЕТРЫ:")
best_params = random_search_mlp.best_params_
for param, value in best_params.items():
    print(f"  {param}: {value}")

Fitting 3 folds for each of 30 candidates, totalling 90 fits

ЛУЧШИЕ ПАРАМЕТРЫ:
  mlp__solver: adam
  mlp__max_iter: 10
  mlp__learning_rate_init: 0.001
  mlp__hidden_layer_sizes: (20, 10)
  mlp__alpha: 0.0001
  mlp__activation: relu


/usr/local/lib/python3.12/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


## Лучшая модель

In [26]:
best_model = MLPClassifier(
    random_state=42,
    verbose=0,
    early_stopping=True,  # ранняя остановка
    validation_fraction=0.1,  # доля данных для валидации при early_stopping
    n_iter_no_change=10,  # остановка если нет улучшения 10 эпох
    tol=1e-4,  # порог для сходимости
    solver='adam',
    max_iter=100,
    learning_rate_init=0.001,
    hidden_layer_sizes=(100, 50),
    alpha=0.0001,
    activation='relu'
)

best_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('mlp', best_model)
])

In [27]:
best_pipeline.fit(X_train, y_train)

y_val_pred_proba = best_pipeline.predict_proba(X_val)[:, 1]
val_ap_score = average_precision_score(y_val, y_val_pred_proba)
print(f"\nAverage Precision Score: {val_ap_score:.4f}")


Average Precision Score: 0.8002


In [28]:
test_prediction = best_pipeline.predict_proba(X_test)[:, 1]
sub_df = test_df.select("id").collect().to_pandas()
sub_df.loc[:, "y_pred"] = test_prediction
sub_df.to_csv("/content/drive/MyDrive/wb/mlp_submission.csv", index=False)
sub_df.head()

,id,y_pred
0,82e68c9404e40d1c3577,0.367730
1,3f9db954a1926f8e58e8,0.396183
2,a46ae482ef6932a2b95d,0.960326
3,eba9fec0558ac6f80602,0.889222
4,2725e3d5cd74a31d08a7,0.866701
